In [ ]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
house_precinct_csv = 'data/raw/HOUSE_precinct_general.csv'
dime_csv = 'data/raw/dime.csv'
census_mi_vtd_csv = 'data/MI_VTD_demographics.csv'
census_house_mi_csv = 'data/mi_match_vtd_house.csv'

In [ ]:
#HOUSE official dtypes from documentation
official_dtypes = {'precinct':str,'office':str, 'party_detailed':str, 'party_simplified':str,
'mode':str,'votes':int, 'county_name':str, 'county_fips':str, 'jurisdiction_name':str,
'jurisdiction_fips':str, 'candidate':str, 'district':str, 'dataverse':str,'year':int,
'stage':str, 'state':str, 'special':str, 'writein':str, 'state_po':str, 'state_fips':str,
'state_cen':str, 'state_ic':str, 'date':str, 'readme_check':str,'magnitude':int}

house =  pd.read_csv(house_precinct_csv, dtype = official_dtypes)

In [ ]:
#Filter states
states = ['MI']
house = house[house['state_po'].isin(states)]

In [ ]:
#Filter office to US House only
house = house[house['office'].isin(['US HOUSE'])]

In [ ]:
#Filter precincts with zero votes
house = house[house['votes']!=0]

In [ ]:
#Filter Statistical Adjustments
house = house[house['jurisdiction_name']!="{STATISTICAL ADJUSTMENTS}"]

In [ ]:
# Michigan also reports the 'absentee' vote (mode == 'ABSENTEE')
# Is unclear how to match the absentee ballot votes to a precinct
# For this reason, I am dropping them here.
# We should check how this might affect our analysis.
house = house[house['mode']!='ABSENTEE']

In [ ]:
#Concatenate state and district to match district column in DIME
house['state_po_district'] = house['state_po'] + house['district'].str[1:]

In [ ]:
#Each state reports precincts differently
#The following is only for MI
house[['precinct_code', 'ward_code']] = house['precinct'].str.split(',', expand = True)
house['precinct_code'] = house['precinct_code'].apply('{0:0>3}'.format)
house['ward_code'] = house['ward_code'].str.replace('WARD','')
house['ward_code'] = house['ward_code'].str.strip()
house['ward_code'] = house['ward_code'].fillna('0')
house['ward_code'] = house['ward_code'].apply('{0:0>2}'.format)

In [ ]:
#Note: We still need to check the precincts that have code with letters like '0-22L', '1-A', '1-B' for MI
#Dropping them from now
house['tocheck'] = np.where(house.precinct_code.str.contains('-'), '1', '0')
house['precinct'][(house['tocheck'] == '1')]
house = house[house['tocheck']!='1']

In [ ]:
#Create NAME20 to match census data
#In census data NAME20 = COUNTYFIPS + MCDFIPS + WARD + PRECINCT
#In house data we have jurisdiction_fips = state_fips + county_fips + mcd_fips
#and we created ward_code and precinct_code
#so we have to remove state_fips from jurisdiction_fips and add ward_code and precinct_code
house['NAME20'] = house['jurisdiction_fips'].str[2:] + house['ward_code'] + house['precinct_code']

In [ ]:
#Edit candidate names based on inspection
house['candidate']  = house['candidate'] .str.replace('FRED UPTON','FREDERICK STEPHEN UPTON')

In [ ]:
#Totals by precinct for MI
total_by_prec_cand = house.groupby(['state_po', 'state_po_district', 'NAME20','candidate'])['votes'].sum().reset_index()
total_by_prec_cand


In [ ]:
total_by_prec = total_by_prec_cand.groupby(['state_po', 'state_po_district', 'NAME20'])['votes'].sum().reset_index()
total_by_prec = total_by_prec.rename(columns = {'votes': 'total_votes'})
total_by_prec

In [ ]:
result = pd.merge(total_by_prec_cand, total_by_prec, on = ['state_po', 'state_po_district', 'NAME20'])
result['pct_vote'] = result['votes']/result['total_votes']
result['candidate_house'] = result['candidate'].str.lower()
result['candidate_house'] = result['candidate_house'].str.replace('.', '')
result['candidate_house'] = result['candidate_house'].str.strip()
result

In [ ]:
#Commenting the lines where we computed the total by county for the second deliverable
#Totals by county
#total_by_county_cand = house.groupby(['state_po', 'state_po_district', 'county_fips','candidate'])['votes'].sum().reset_index()
#total_by_county = total_by_county_cand.groupby(['state_po', 'state_po_district', 'county_fips'])['votes'].sum().reset_index()
#total_by_county = total_by_county.rename(columns = {'votes': 'total_votes'})

In [ ]:
#result = pd.merge(total_by_county_cand, total_by_county, on = ['state_po', 'state_po_district', 'county_fips'])
#result['pct_vote'] = result['votes']/result['total_votes']
#result['candidate_house'] = result['candidate'].str.lower()
#result['candidate_house'] = result['candidate_house'].str.replace('.', '')
#result['candidate_house'] = result['candidate_house'].str.strip()

In [ ]:
#DIME
dime =  pd.read_csv(dime_csv)
dime.sort_values(by=['district', 'name'])
dime = dime.drop(columns=['cycle', 'seat', 'ran_general', 'winner', 'recipient_type'])

In [ ]:
# new data frame with split name
split = dime['name'].str.split(",", n = 1, expand = True)

In [ ]:
# Change party number codes to letter codes
dime.loc[dime['party'] == 100, 'party'] = 'D'
dime.loc[dime['party'] == 200, 'party'] = 'R'
dime.loc[dime['party'] == 328, 'party'] = 'I'

In [ ]:
#Create new candidate name to match with HOUSE df, and new id column to match with census
dime['candidate_dime'] = dime['ffname'] + " " + split[0]
dime['candidate_dime'] = dime['candidate_dime'].str.replace('.', '')
dime['candidate_dime'] = dime['candidate_dime'].str.replace('mr', '')
dime['candidate_dime'] = dime['candidate_dime'].str.strip()

dime['census_match'] = dime['party'] + dime['name'].astype(str).str[:3] + dime['state']

In [ ]:
def checker(wrong_options,correct_options):
    names_array=[]
    ratio_array=[]    
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
           names_array.append(wrong_option)
           ratio_array.append('100')
        else:   
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_set_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])
    return names_array,ratio_array

In [ ]:
result_MI = result[result['state_po'].isin(['MI'])]
result_MI = result_MI['candidate_house'].drop_duplicates()
dime_MI = dime[dime['state'].isin(['MI'])]

str2Match = result_MI.tolist()
strOptions = dime_MI['candidate_dime'].fillna('######').tolist()

name_match,ratio_match=checker(str2Match,strOptions)
merged_MI = pd.DataFrame()
merged_MI['candidate_house'] = pd.Series(str2Match)
merged_MI['candidate_dime'] = pd.Series(name_match)
merged_MI['correct_ratio'] = pd.Series(ratio_match).astype(float)

merged_MI.sort_values(by=['correct_ratio'])

In [ ]:
#merge with pivots of fuzzy matching
result = result.merge(merged_MI, how = 'left', on = 'candidate_house')
result

In [ ]:
# merge with dime data to add ideology
result = result.merge(dime, how = 'left', on = 'candidate_dime')
result

In [ ]:
# Calculate the min correct ratio by precinct
result = result.join(result.groupby(['state_po_district', 'NAME20'])['correct_ratio'].agg(['min']), on = ['state_po_district', 'NAME20'])

# Keep only precincts where min correct ratio is at least 70
result = result[result['min']>=70]
result


In [ ]:
#Pivot data from long to wide
result = result[['state_po_district', 'NAME20', 'candidate_house',
                'candidate_dime', 'pct_vote', 'party',
                'recipient_cfscore', 'wgt_cfscore', 'cf_category', 'cf_label']]

In [ ]:
pivot = result.pivot_table(index=['NAME20'], 
                            columns=['party'])
pivot.columns = ["_".join((j,i)) for i,j in pivot.columns]
pivot

In [ ]:
# Compute ideology score weighted by % votes
#result['wgt_cfscore'] = result['recipient_cfscore'] * result['pct_vote']
#result

In [ ]:
for x in ['D', 'I', 'R']:
    score = x + '_recipient_cfscore'
    cat = x + '_cf_category'
    label = x + '_cf_label'

    pivot[cat] = ''
    pivot.loc[(pivot[score] < -1), cat] = '-3'
    pivot.loc[(pivot[score] >= -1) & (pivot[score] <= -0.5), cat] = '-2'
    pivot.loc[(pivot[score] >= -0.5) & (pivot[score] < 0), cat] = '-1'
    pivot.loc[(pivot[score] >= 0) & (pivot[score] < 0.5), cat] = '1'
    pivot.loc[(pivot[score] >= 0.5) & (pivot[score] < 1), cat] = '2'
    pivot.loc[(pivot[score] >= 1), cat] = '3'

    pivot[label] = ''
    pivot.loc[(pivot[cat] == '-3'), label] = 'Very Liberal'
    pivot.loc[(pivot[cat] == '-2'), label] = 'Strong Liberal'
    pivot.loc[(pivot[cat] == '-1'), label] = 'Lean Liberal'
    pivot.loc[(pivot[cat] == '1'), label] = 'Lean Conservative'
    pivot.loc[(pivot[cat] == '2'), label] = 'Strong Conservative'
    pivot.loc[(pivot[cat] == '3'), label] = 'Very Conservative'
    
pivot

In [53]:
#Map Census to House data
map_census_house = pd.read_csv(census_house_mi_csv, dtype = {'GEOID20':str, 'NAME20':str, 'jurisdiction_fips':str})
map_census_house_mi = map_census_house[['GEOID20', 'NAME20']]
map_census_house_mi['VTD_ID'] = map_census_house_mi['GEOID20']

/var/folders/sn/cp78km453yq1y8g3c822ww7r0000gn/T/ipykernel_11593/2724068784.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_census_house_mi['VTD_ID'] = map_census_house_mi['GEOID20']


In [54]:
mi_result = pivot.merge(map_census_house_mi, how = 'left', on = 'NAME20')

In [55]:
mi_result

,NAME20,D_pct_vote,I_pct_vote,R_pct_vote,D_recipient_cfscore,I_recipient_cfscore,R_recipient_cfscore,D_wgt_cfscore,I_wgt_cfscore,R_wgt_cfscore,...,Rcf_category,Rcf_label,D_cf_category,D_cf_label,I_cf_category,I_cf_label,R_cf_category,R_cf_label,GEOID20,VTD_ID
0,0010104000001,0.322775,NaN,0.677225,-1.474,NaN,1.086,-0.475771,NaN,0.735466,...,3,Very Conservative,-3,Very Liberal,,,3,Very Conservative,26001001001,26001001001
1,0011246000001,0.314236,NaN,0.685764,-1.474,NaN,1.086,-0.463184,NaN,0.744740,...,3,Very Conservative,-3,Very Liberal,,,3,Very Conservative,26001001002,26001001002
2,0011932000001,0.350534,NaN,0.649466,-1.474,NaN,1.086,-0.516687,NaN,0.705320,...,3,Very Conservative,-3,Very Liberal,,,3,Very Conservative,26001001003,26001001003
3,0013482000001,0.360000,NaN,0.640000,-1.474,NaN,1.086,-0.530640,NaN,0.695040,...,3,Very Conservative,-3,Very Liberal,,,3,Very Conservative,26001001004,26001001004
4,0013574000001,0.274834,NaN,0.725166,-1.474,NaN,1.086,-0.405106,NaN,0.787530,...,3,Very Conservative,-3,Very Liberal,,,3,Very Conservative,26001001005,26001001005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,1636508000008,0.483419,0.010685,0.495210,-1.376,0.5325,1.018,-0.665185,0.009207,0.504124,...,3,Very Conservative,-3,Very Liberal,2,Strong Conservative,3,Very Conservative,26163163816,26163163816
1076,1636508000009,0.498650,0.014851,0.471647,-1.376,0.5325,1.018,-0.686142,0.013158,0.480137,...,3,Very Conservative,-3,Very Liberal,2,Strong Conservative,3,Very Conservative,26163163817,26163163817
1077,1636508000010,0.421380,0.009012,0.560597,-1.376,0.5325,1.018,-0.579819,0.007765,0.570687,...,3,Very Conservative,-3,Very Liberal,2,Strong Conservative,3,Very Conservative,26163163818,26163163818
1078,1636508000011,0.424718,0.008156,0.558971,-1.376,0.5325,1.018,-0.584412,0.007004,0.569033,...,3,Very Conservative,-3,Very Liberal,2,Strong Conservative,3,Very Conservative,26163163819,26163163819


In [56]:
#Census data

mi_vtd = pd.read_csv(census_mi_vtd_csv)
# Drop rows where VTD has ZZZ
remove = mi_vtd['VTD_ID'].str.contains('Z')
mi_vtd = mi_vtd[ ~remove]
mi_vtd

,VTD_ID,total_pop,pop_density,prop_white,prop_Black,prop_Hispanic,prop_AIA,prop_Asian,prop_married,prop_nonfamily,prop_family_nonmarried,prop_hs_diploma,prop_some_college,prop_bachelors,prop_higher,prop_under_hs,prop_under_30k,prop_30_to_50k,prop_50_to_100k,prop_more_than_100k
0,26001001001,1199.706208,0.000020,0.976484,0.001091,0.007787,0.003980,0.003651,0.586734,0.345760,0.067506,0.000781,0.159600,0.146117,0.091453,0.602049,0.272519,0.216670,0.383058,0.127752
1,26001001002,888.447426,0.000014,0.974214,0.000000,0.007050,0.003035,0.004946,0.544382,0.376283,0.079335,0.002868,0.127402,0.131108,0.100085,0.638538,0.390689,0.190452,0.276680,0.142178
2,26001001003,732.583185,0.000008,0.948043,0.000000,0.018285,0.024794,0.008878,0.427789,0.485719,0.086492,0.000000,0.150100,0.076132,0.029278,0.744491,0.425132,0.307724,0.179958,0.087186
3,26001001004,1234.485189,0.000060,0.969429,0.010086,0.003118,0.005656,0.008460,0.521359,0.397147,0.081493,0.000000,0.215217,0.123143,0.028745,0.632895,0.336506,0.257895,0.326713,0.078885
4,26001001005,746.017711,0.000030,0.958360,0.001958,0.016015,0.011473,0.000809,0.410391,0.438630,0.150979,0.000000,0.175692,0.053673,0.040530,0.730105,0.461984,0.228334,0.239768,0.069915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4800,26165165017,2263.702306,0.000096,0.945013,0.004608,0.020610,0.008239,0.022435,0.582442,0.295589,0.121969,0.003405,0.180882,0.116643,0.081976,0.617093,0.235246,0.229657,0.339395,0.195702
4801,26165165018,706.378252,0.000014,0.970497,0.004658,0.022516,0.002329,0.000000,0.541748,0.314563,0.143689,0.000000,0.157110,0.049312,0.030963,0.762615,0.347573,0.283495,0.285437,0.083495
4802,26165165019,499.350584,0.000029,0.966692,0.001116,0.010215,0.017693,0.003213,0.562134,0.311252,0.126614,0.000000,0.124344,0.081335,0.049400,0.744921,0.319067,0.284055,0.338750,0.058128
4803,26165165020,771.989062,0.000046,0.954362,0.008375,0.012670,0.015137,0.007649,0.533309,0.332431,0.134260,0.000000,0.164235,0.068926,0.038507,0.728332,0.349924,0.263310,0.309916,0.076850


In [57]:
mi_result = mi_result.merge(mi_vtd, how = 'left', on = 'VTD_ID')
mi_result

,NAME20,D_pct_vote,I_pct_vote,R_pct_vote,D_recipient_cfscore,I_recipient_cfscore,R_recipient_cfscore,D_wgt_cfscore,I_wgt_cfscore,R_wgt_cfscore,...,prop_family_nonmarried,prop_hs_diploma,prop_some_college,prop_bachelors,prop_higher,prop_under_hs,prop_under_30k,prop_30_to_50k,prop_50_to_100k,prop_more_than_100k
0,0010104000001,0.322775,NaN,0.677225,-1.474,NaN,1.086,-0.475771,NaN,0.735466,...,0.067506,0.000781,0.159600,0.146117,0.091453,0.602049,0.272519,0.216670,0.383058,0.127752
1,0011246000001,0.314236,NaN,0.685764,-1.474,NaN,1.086,-0.463184,NaN,0.744740,...,0.079335,0.002868,0.127402,0.131108,0.100085,0.638538,0.390689,0.190452,0.276680,0.142178
2,0011932000001,0.350534,NaN,0.649466,-1.474,NaN,1.086,-0.516687,NaN,0.705320,...,0.086492,0.000000,0.150100,0.076132,0.029278,0.744491,0.425132,0.307724,0.179958,0.087186
3,0013482000001,0.360000,NaN,0.640000,-1.474,NaN,1.086,-0.530640,NaN,0.695040,...,0.081493,0.000000,0.215217,0.123143,0.028745,0.632895,0.336506,0.257895,0.326713,0.078885
4,0013574000001,0.274834,NaN,0.725166,-1.474,NaN,1.086,-0.405106,NaN,0.787530,...,0.150979,0.000000,0.175692,0.053673,0.040530,0.730105,0.461984,0.228334,0.239768,0.069915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,1636508000008,0.483419,0.010685,0.495210,-1.376,0.5325,1.018,-0.665185,0.009207,0.504124,...,0.062580,0.012191,0.099239,0.299268,0.308993,0.280310,0.099188,0.132524,0.232912,0.535376
1076,1636508000009,0.498650,0.014851,0.471647,-1.376,0.5325,1.018,-0.686142,0.013158,0.480137,...,0.177105,0.000000,0.157408,0.260273,0.212850,0.369469,0.165269,0.170149,0.302305,0.362277
1077,1636508000010,0.421380,0.009012,0.560597,-1.376,0.5325,1.018,-0.579819,0.007765,0.570687,...,0.062453,0.000000,0.119984,0.335917,0.360532,0.183567,0.082955,0.070514,0.273983,0.572548
1078,1636508000011,0.424718,0.008156,0.558971,-1.376,0.5325,1.018,-0.584412,0.007004,0.569033,...,0.025579,0.000000,0.057533,0.276987,0.424674,0.240807,0.000000,0.096224,0.160780,0.742996


In [58]:
mi_result.columns

Index(['NAME20', 'D_pct_vote', 'I_pct_vote', 'R_pct_vote',
       'D_recipient_cfscore', 'I_recipient_cfscore', 'R_recipient_cfscore',
       'D_wgt_cfscore', 'I_wgt_cfscore', 'R_wgt_cfscore', 'Dcf_category',
       'Dcf_label', 'Icf_category', 'Icf_label', 'Rcf_category', 'Rcf_label',
       'D_cf_category', 'D_cf_label', 'I_cf_category', 'I_cf_label',
       'R_cf_category', 'R_cf_label', 'GEOID20', 'VTD_ID', 'total_pop',
       'pop_density', 'prop_white', 'prop_Black', 'prop_Hispanic', 'prop_AIA',
       'prop_Asian', 'prop_married', 'prop_nonfamily',
       'prop_family_nonmarried', 'prop_hs_diploma', 'prop_some_college',
       'prop_bachelors', 'prop_higher', 'prop_under_hs', 'prop_under_30k',
       'prop_30_to_50k', 'prop_50_to_100k', 'prop_more_than_100k'],
      dtype='object')

In [ ]:
mi_result.dropna(inplace=True)
mi_result = mi_result[['NAME20','VTD_ID', 'total_pop',
       'pop_density', 'prop_white', 'prop_Black', 'prop_Hispanic', 'prop_AIA',
       'prop_Asian', 'prop_married', 'prop_nonfamily',
       'prop_family_nonmarried', 'prop_hs_diploma', 'prop_some_college',
       'prop_bachelors', 'prop_higher', 'prop_under_hs', 'prop_under_30k',
       'prop_30_to_50k', 'prop_50_to_100k', 'prop_more_than_100k',
       'D_pct_vote', 'I_pct_vote', 'R_pct_vote',
       'D_recipient_cfscore', 'I_recipient_cfscore', 'R_recipient_cfscore',
       'D_cf_category', 'D_cf_label', 'I_cf_category', 'I_cf_label',
       'R_cf_category', 'R_cf_label']]

In [ ]:
#Export to csv
mi_result.to_csv('data/mi_matched_precinct.csv', index = False)

In [ ]:
#result_agg = result.groupby(['state_po_district', 'county_fips'])['wgt_cfscore'].sum().reset_index()
#result_agg

In [ ]:
#Lean Liberal (0 through -0.5)
#Strong Liberal (-0.5 through -1)
#Very Liberal (-1 and lower)
#Lean Conservative (0 through 0.5)
#Strong Conservative (0.5 through 1)
#Very Conservative (1 and higher)
# result_agg['cf_category'] = ''
# result_agg.loc[(result_agg['wgt_cfscore'] < -1), 'cf_category'] = '-3'
# result_agg.loc[(result_agg['wgt_cfscore'] >= -1) & (result_agg['wgt_cfscore'] <= -0.5), 'cf_category'] = '-2'
# result_agg.loc[(result_agg['wgt_cfscore'] >= -0.5) & (result_agg['wgt_cfscore'] < 0), 'cf_category'] = '-1'
# result_agg.loc[(result_agg['wgt_cfscore'] >= 0) & (result_agg['wgt_cfscore'] < 0.5), 'cf_category'] = '1'
# result_agg.loc[(result_agg['wgt_cfscore'] >= 0.5) & (result_agg['wgt_cfscore'] < 1), 'cf_category'] = '2'
# result_agg.loc[(result_agg['wgt_cfscore'] >= 1), 'cf_category'] = '3'

# result_agg['cf_label'] = ''
# result_agg.loc[(result_agg['cf_category'] == '-3'), 'cf_label'] = 'Very Liberal'
# result_agg.loc[(result_agg['cf_category'] == '-2'), 'cf_label'] = 'Strong Liberal'
# result_agg.loc[(result_agg['cf_category'] == '-1'), 'cf_label'] = 'Lean Liberal'
# result_agg.loc[(result_agg['cf_category'] == '1'), 'cf_label'] = 'Lean Conservative'
# result_agg.loc[(result_agg['cf_category'] == '2'), 'cf_label'] = 'Strong Conservative'
# result_agg.loc[(result_agg['cf_category'] == '3'), 'cf_label'] = 'Very Conservative'

# result_agg

In [ ]:
#export to csv
#result_agg.to_csv('data/ideology_county.csv', index = False)

In [ ]:
#mi_result.dropna(inplace=True)
#export to csv
#mi_result.to_csv('data/mi_matched_county.csv', index = False)

In [ ]:
# for x in ['prop_white', 'prop_Black', 'prop_Hispanic', 'prop_AIA', 'prop_Asian']:
#     mi_vtd[x] = mi_vtd['total_pop'] * mi_vtd[x]
# mi_vtd

In [ ]:
# mi_vtd['county_fips'] = mi_vtd['VTD_ID'].str[:5]
# mi_vtd

In [ ]:
# mi_county = mi_vtd.groupby(['county_fips']).agg(
#      total_pop = ('total_pop','sum'),
#      total_White = ('prop_white','sum'),
#      total_Black = ('prop_Black','sum'),
#      total_Hispanic = ('prop_Hispanic','sum'),
#      total_AIA = ('prop_AIA','sum'),
#      total_Asian = ('prop_Asian','sum'),
#      ).reset_index()
# mi_county

In [ ]:
# mi_county['prop_White'] = mi_county['total_White'] / mi_county['total_pop']
# mi_county['prop_Black'] = mi_county['total_Black'] / mi_county['total_pop']
# mi_county['prop_Hispanic'] = mi_county['total_Hispanic'] / mi_county['total_pop']
# mi_county['prop_AIA'] = mi_county['total_AIA'] / mi_county['total_pop']
# mi_county['prop_Asian'] = mi_county['total_Asian'] / mi_county['total_pop']
# mi_county

In [ ]:
# mi_result = mi_county.merge(result_agg, how = 'left', on = 'county_fips')
# mi_result

In [ ]:
#mi_result = mi_result[['county_fips','total_pop','prop_White','prop_Black','prop_Hispanic','prop_AIA','prop_Asian', 'wgt_cfscore','cf_category','cf_label']]
